# Logistic Regression Consulting Project

## Binary Customer Churn
Predicting which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company


In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 52 kB/s 
     |████████████████████████████████| 199 kB 47.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=8b2bb4841d9363e6c5fd48e0453ddc13f9598de17ca6ad516e64477c64f23d6f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [5]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
final_data = output.select('features','churn')

In [13]:
#Test Train Split

In [14]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

Model building

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
lr_churn = LogisticRegression(labelCol='churn')

In [17]:
fitted_churn_model = lr_churn.fit(train_churn)

In [21]:
fitted_churn_model.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                601|                601|
|   mean|0.17304492512479203|0.11980033277870217|
| stddev|0.37860121453190837| 0.3249984000984554|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



Evaluation

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [25]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.70265421411717...|[0.99101037807548...|       0.0|
|[25.0,9672.03,0.0...|    0|[4.77714250615541...|[0.99165028005561...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.93338852212254...|[0.71776223721538...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.75383933232964...|[0.85243639991785...|       0.0|
|[28.0,11245.38,0....|    0|[3.86738901077696...|[0.97951548899871...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.86766849111246...|[0.94622483625456...|       0.0|
|[29.0,10203.18,1....|    0|[3.8458487661416,...|[0.97907879253339...|       0.0|
|[29.0,12711.15,0....|    0|[5.27784339302082...|[0.99492248768801...|       0.0|
|[29.0,13240.01,1....|    0|[6.48188020780049...|[0.99847141065655...|       0.0|
|[30.0,11575.37,

Using AUC

In [26]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [27]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [28]:
auc

0.8300395256916996

**Predict on brand new unlabeled data**

We still need to evaluate the new_customers.csv file!

In [29]:
final_lr_model = lr_churn.fit(final_data)

In [30]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [31]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [32]:
test_new_customers = assembler.transform(new_customers)

In [33]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [34]:
final_results = final_lr_model.transform(test_new_customers)

In [35]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

